# Feature Engineering

## Что такое пропущенные значения
Аналитикам часто приходится работать с данными, в которых есть пропуски. Более того, в реальных данных пропуски сплошь и рядом.

Когда могут появиться пропуски в данных? Например, если мы делаем опрос, люди могут просто не ответить на какие-то из вопросов. В этих местах появляется пропуск. Еще один пример — отправляем данные по протоколу UDP, часть данных теряется — снова пропуски. Ну и самый банальный случай — программа заглючила и не записала часть данных :).

Чтобы программа правильно интерпретировала пропуски, при чтении файла с помощью метода read_csv можно передать в параметр na_values значение или список значений, которые при чтении будут помечены как пропуски.

## Какие бывают пропуски
Вот список значений, которые по умолчанию считаются как пропуски: '', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', '1.#QNAN', 'N/A', 'NA', 'NULL', 'NaN', 'n/a', 'nan', 'null'.  

Посмотрите внимательно на список значений выше: можно заметить, что первым значением в списке является пустая строка '' - это именно тот случай, когда значения в данных просто нет.

In [1]:
import pandas as pd

In [2]:
users_df = pd.read_csv('users.csv', sep='\t', encoding='KOI8-R')

In [6]:
users_df.head()

,Юзверь,мейл,Гео
0,User_943,Accumanst@gmail.com,Ижевск
1,User_908,Advismowr@mail.ru,Ижевск
2,User_962,Anachso@ukr.net,Краснодар
3,User_973,Antecia@inbox.ru,Пермь
4,User_902,Balliaryva@ukr.net,NaN


In [7]:
users_df.columns = ['user_id', 'email', 'geo']

In [11]:
users_df.head()

,user_id,email,geo
0,User_943,Accumanst@gmail.com,Ижевск
1,User_908,Advismowr@mail.ru,Ижевск
2,User_962,Anachso@ukr.net,Краснодар
3,User_973,Antecia@inbox.ru,Пермь
4,User_902,Balliaryva@ukr.net,NaN


In [8]:
log_df = pd.read_csv('log.csv', header=None)

In [9]:
log_df.head()

,0,1,2,3
0,Запись пользователя № - user_919,[2019-01-01 14:06:51,NaN,NaN
1,Запись пользователя № - user_973,[2019-01-01 14:51:16,NaN,NaN
2,Запись пользователя № - user_903,[2019-01-01 16:31:16,NaN,NaN
3,Запись пользователя № - user_954,[2019-01-01 17:17:51,NaN,NaN
4,Запись пользователя № - user_954,[2019-01-01 21:31:18,NaN,NaN


In [10]:
log_df.columns = ['user_id', 'time', 'bet', 'win']

In [12]:
log_df.head()

,user_id,time,bet,win
0,Запись пользователя № - user_919,[2019-01-01 14:06:51,NaN,NaN
1,Запись пользователя № - user_973,[2019-01-01 14:51:16,NaN,NaN
2,Запись пользователя № - user_903,[2019-01-01 16:31:16,NaN,NaN
3,Запись пользователя № - user_954,[2019-01-01 17:17:51,NaN,NaN
4,Запись пользователя № - user_954,[2019-01-01 21:31:18,NaN,NaN


## Как найти пропуски
В pandas есть метод `isna()`, который возвращает таблицу такой же размерности, что и на вход, но значения в ней - True или False. True, если данное значение является пропуском, и False в ином случае.

In [13]:
log_df.head().isna()

,user_id,time,bet,win
0,False,False,True,True
1,False,False,True,True
2,False,False,True,True
3,False,False,True,True
4,False,False,True,True


Зачем нам смотреть на эту таблицу? На самом деле, незачем, но с помощью нее можно делать полезные вещи. Например, считать количество пропусков и работать с ними, чем мы и займемся дальше.

В numpy пропущенные значения могут быть записаны как специальный объект np.nan, что означает Not a Number.

Проверить на наличие таких значений можно с помощью np.isnan().

P.S. Если вы все-таки хотите посмотреть на красивую (и, возможно, полезную) визуализацию пропущенных значений - обратите внимание на библиотеку [missingno](https://github.com/ResidentMario/missingno).

#### Посчитайте количество пропусков в столбце time. Метод isna() есть не только у DataFrame, но и у Series. Это значит, что применять его можно не только ко всей таблице, но и к каждому столбцу отдельно.

In [17]:
len(log_df[log_df['time'].isna()])

15